# I. Introduction to Cohort Analysis

## 1.1. Overall Understanding to Cohort Analysis

- Definition: 
    + Cohort analysis is a method of customer segmentation that help businesses understand their customers. Cohort analysis focuses on how a customer's behavior changes over time, while RFM analysis focuses on a customer's current behavior. 
- Focus: 
    + How a customer's behavior changes over time 
- Goal: 
    + Identify trends and patterns in customer behavior 
    + Compare metrics across product lifecycle
    + Compare metrics across customer lifecycle
- How it works: 
    + Groups customers into mutually exclusive **cohorts** based on when they were acquired and tracks their behavior over time 
- When it's useful: 
    + For understanding how a group of customers evolves over time
- Benefits of cohort analysis:
    + Helps identify trends and patterns in customer behavior
    + Helps identify key metrics like retention rate and upsell rate
    + Can be used to improve customer experience and retention

## 1.2. Types of Cohorts

- **Time Cohorts:**
    + Time Cohorts are customers who signed up for a product or service during a particular time frame. 
    + Analyzing these cohorts shows the customers’ behavior depending on the time they started using the company’s products | services. 
    + The time may be daily or weekly or monthly or quarterly.

- **Behavior Cohorts:**
    + Behavior Cohorts are customers who purchased a product or subscribed to a service in the past. 
    + It groups customers by the type of product or service they signed up. 
    + Customers who signed up for basic level services might have different needs than those who signed up for advanced services. 
    + Understanding the needs of the various cohorts can help a company design custom-made services or products for particular segments.
    
- **Size Cohorts:**
    + Size Cohorts refer to the various sizes of customers who purchase company’s products or services. 
    + This categorization can be based on the amount of spending in some period of time after acquisition, or the product type that the customer spent most of their order amount in some period of time.

## 1.3. Time Cohort (Customer Cohort) Example

### Load packages

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff

### Load data 

In [3]:
# load csv file as a DataFrame. The encoding is required to read the file
online = pd.read_csv('data/E-Commerce Data.csv', encoding='ISO-8859-1')

In [4]:
# # Load xlsx file as a DataFrame
# second_online = pd.read_excel(
#     'data/online_retail_II.xlsx'
#     # , sheet_name='Year 2009-2010'
#     , sheet_name='Year 2010-2011'
# )

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### Data Pre-processing

In [1]:
# include only UK
online = online[online['Country'] == 'United Kingdom']

# include only date, price, customerid 
online = online[['InvoiceDate','UnitPrice','CustomerID']]

# remove duplicates
online.drop_duplicates(inplace=True)

# Drop rows with missing CustomerID
online = online[~online['CustomerID'].isnull()]
online.reset_index(drop=True, inplace=True)

# datetime conversion
online['InvoiceDate'] = pd.to_datetime(online['InvoiceDate'],format='%m/%d/%Y %H:%M')

# view first 5 rows
online.head()


,InvoiceDate,UnitPrice,CustomerID
0,2010-12-01 08:26:00,2.55,17850.0
1,2010-12-01 08:26:00,3.39,17850.0
2,2010-12-01 08:26:00,2.75,17850.0
3,2010-12-01 08:26:00,7.65,17850.0
4,2010-12-01 08:26:00,4.25,17850.0


In [2]:
# preview data information
online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159858 entries, 0 to 159857
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceDate  159858 non-null  datetime64[ns]
 1   UnitPrice    159858 non-null  float64       
 2   CustomerID   159858 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 3.7 MB


In [3]:
# get first day of the month from a given date object
def get_month(datetime: dt.datetime) -> dt.datetime:
    '''get first day of the month from a given date object: datetime64[ns]'''
    return dt.datetime(
        year=datetime.year
        ,month=datetime.month
        ,day=1
    )

# assign acquisition month cohort
online['InvoiceMonth'] = online['InvoiceDate'].apply(get_month)

# preview first 5 rows
online.head()

,InvoiceDate,UnitPrice,CustomerID,InvoiceMonth
0,2010-12-01 08:26:00,2.55,17850.0,2010-12-01
1,2010-12-01 08:26:00,3.39,17850.0,2010-12-01
2,2010-12-01 08:26:00,2.75,17850.0,2010-12-01
3,2010-12-01 08:26:00,7.65,17850.0,2010-12-01
4,2010-12-01 08:26:00,4.25,17850.0,2010-12-01


In [4]:
# assign the smallest InvoiceMonth value of each customer to each record regarding the same customer ID
online['CohortMonth'] = online.groupby('CustomerID')['InvoiceMonth'].transform('min')

# preview last 5 rows
online.tail()

,InvoiceDate,UnitPrice,CustomerID,InvoiceMonth,CohortMonth
159853,2011-12-09 12:31:00,0.95,15804.0,2011-12-01,2011-05-01
159854,2011-12-09 12:49:00,2.95,13113.0,2011-12-01,2010-12-01
159855,2011-12-09 12:49:00,1.25,13113.0,2011-12-01,2010-12-01
159856,2011-12-09 12:49:00,8.95,13113.0,2011-12-01,2010-12-01
159857,2011-12-09 12:49:00,7.08,13113.0,2011-12-01,2010-12-01


In [5]:
# Assign Time Offset Value Process

# def helper function
def get_date_as_int(dataframe: pd.DataFrame, column: str) -> tuple:
    '''
    Function to extract year, month and day as integer values from a datetime object. 
    
    Required pandas as pd and datetime as dt packages

    Return tuple(year: Series[int], month: Series[int], day: Series[int])
    '''
    year = dataframe[column].dt.year
    month = dataframe[column].dt.month
    day = dataframe[column].dt.day
    return year, month, day

# extract year, month from InvoiceMonth, CohortMonth variables
invoice_year, invoice_month, _ = get_date_as_int(online,'InvoiceMonth')
cohort_year, cohort_month, _ = get_date_as_int(online,'CohortMonth')

# Calculate time offset including year and month
years_diff = invoice_year - cohort_year
months_diff = invoice_month - cohort_month

# create cohort index
online['CohortIndex'] = years_diff * 12 + months_diff

# excluding unnecessary columns
# online = online[['CustomerID','CohortMonth','CohortIndex']]

# preview last 5 rows
online.tail()

,InvoiceDate,UnitPrice,CustomerID,InvoiceMonth,CohortMonth,CohortIndex
159853,2011-12-09 12:31:00,0.95,15804.0,2011-12-01,2011-05-01,7
159854,2011-12-09 12:49:00,2.95,13113.0,2011-12-01,2010-12-01,12
159855,2011-12-09 12:49:00,1.25,13113.0,2011-12-01,2010-12-01,12
159856,2011-12-09 12:49:00,8.95,13113.0,2011-12-01,2010-12-01,12
159857,2011-12-09 12:49:00,7.08,13113.0,2011-12-01,2010-12-01,12


In [6]:
# understanding cohort table , 
online[online['CustomerID'] == 13110.0].groupby(['InvoiceMonth','CohortMonth','CohortIndex'])['CustomerID'].count().reset_index()

,InvoiceMonth,CohortMonth,CohortIndex,CustomerID
0,2011-02-01,2011-02-01,0,11
1,2011-03-01,2011-02-01,1,26
2,2011-07-01,2011-02-01,5,15
3,2011-10-01,2011-02-01,8,15
4,2011-11-01,2011-02-01,9,11


In [7]:
# calculate monthly active customers from each cohort == Count the number of unique values per customer ID
cohort_data = online.groupby(['CohortMonth','CohortIndex'])['CustomerID'].apply(pd.Series.nunique).reset_index()
cohort_data

,CohortMonth,CohortIndex,CustomerID
0,2010-12-01,0,871
1,2010-12-01,1,322
2,2010-12-01,2,291
3,2010-12-01,3,329
4,2010-12-01,4,308
...,...,...,...
86,2011-10-01,1,86
87,2011-10-01,2,40
88,2011-11-01,0,296
89,2011-11-01,1,41


### Result

In [8]:
# time cohorts table
cohort_counts = cohort_data.pivot(
    index='CohortMonth'
    ,columns='CohortIndex'
    ,values='CustomerID'
)
cohort_counts

CohortIndex,0,1,2,3,4,5,6,7,8,9,10,11,12
CohortMonth,,,,,,,,,,,,,
2010-12-01,871.0,322.0,291.0,329.0,308.0,345.0,327.0,304.0,306.0,346.0,320.0,429.0,238.0
2011-01-01,362.0,84.0,101.0,89.0,124.0,106.0,95.0,94.0,114.0,127.0,131.0,54.0,NaN
2011-02-01,339.0,85.0,65.0,95.0,96.0,86.0,88.0,96.0,94.0,106.0,33.0,NaN,NaN
2011-03-01,408.0,79.0,107.0,88.0,95.0,70.0,107.0,97.0,119.0,38.0,NaN,NaN,NaN
2011-04-01,276.0,62.0,61.0,60.0,57.0,64.0,64.0,73.0,23.0,NaN,NaN,NaN,NaN
2011-05-01,252.0,58.0,43.0,43.0,54.0,60.0,67.0,25.0,NaN,NaN,NaN,NaN,NaN
2011-06-01,207.0,44.0,34.0,51.0,53.0,67.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,172.0,35.0,33.0,40.0,48.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,140.0,37.0,32.0,36.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# DataCamp data, same logic but we use this dataset instead for data consistency
cohort_counts = pd.read_csv('data/cohort_counts.csv',index_col=0)
cohort_counts

,0,1,2,3,4,5,6,7,8,9,10,11,12
CohortMonth,,,,,,,,,,,,,
2010-12-01,716,246.0,221.0,251.0,245.0,285.0,249.0,236.0,240.0,265.0,254.0,348.0,172.0
2011-01-01,332,69.0,82.0,81.0,110.0,90.0,82.0,86.0,104.0,102.0,124.0,45.0,NaN
2011-02-01,316,58.0,57.0,83.0,85.0,74.0,80.0,83.0,86.0,95.0,28.0,NaN,NaN
2011-03-01,388,63.0,100.0,76.0,83.0,67.0,98.0,85.0,107.0,38.0,NaN,NaN,NaN
2011-04-01,255,49.0,52.0,49.0,47.0,52.0,56.0,59.0,17.0,NaN,NaN,NaN,NaN
2011-05-01,249,40.0,43.0,36.0,52.0,58.0,61.0,22.0,NaN,NaN,NaN,NaN,NaN
2011-06-01,207,33.0,26.0,41.0,49.0,62.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,173,28.0,31.0,38.0,44.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,139,30.0,28.0,35.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Question: How many customers have made their first transaction in January 2011 ?

In [10]:
cohort_counts.loc[
    # transaction date
    '2011-01-01'
    # the first transaction is the first column
    ,'0'
]

332

## 1.4. Cohort Metrics

### 1.4.1. Customer Retention Rate

In [11]:
cohort_counts

,0,1,2,3,4,5,6,7,8,9,10,11,12
CohortMonth,,,,,,,,,,,,,
2010-12-01,716,246.0,221.0,251.0,245.0,285.0,249.0,236.0,240.0,265.0,254.0,348.0,172.0
2011-01-01,332,69.0,82.0,81.0,110.0,90.0,82.0,86.0,104.0,102.0,124.0,45.0,NaN
2011-02-01,316,58.0,57.0,83.0,85.0,74.0,80.0,83.0,86.0,95.0,28.0,NaN,NaN
2011-03-01,388,63.0,100.0,76.0,83.0,67.0,98.0,85.0,107.0,38.0,NaN,NaN,NaN
2011-04-01,255,49.0,52.0,49.0,47.0,52.0,56.0,59.0,17.0,NaN,NaN,NaN,NaN
2011-05-01,249,40.0,43.0,36.0,52.0,58.0,61.0,22.0,NaN,NaN,NaN,NaN,NaN
2011-06-01,207,33.0,26.0,41.0,49.0,62.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,173,28.0,31.0,38.0,44.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,139,30.0,28.0,35.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
cohort_size = cohort_counts.iloc[:,0]
cohort_size

CohortMonth
2010-12-01    716
2011-01-01    332
2011-02-01    316
2011-03-01    388
2011-04-01    255
2011-05-01    249
2011-06-01    207
2011-07-01    173
2011-08-01    139
2011-09-01    279
2011-10-01    318
2011-11-01    291
2011-12-01     38
Name: 0, dtype: int64

In [47]:
retention_percentage_table = cohort_counts.divide(
    other=cohort_size
    ,axis=0
)
retention_percentage_table

,0,1,2,3,4,5,6,7,8,9,10,11,12
CohortMonth,,,,,,,,,,,,,
2010-12-01,1.0,0.343575,0.308659,0.350559,0.342179,0.398045,0.347765,0.329609,0.335196,0.370112,0.354749,0.486034,0.240223
2011-01-01,1.0,0.207831,0.246988,0.243976,0.331325,0.271084,0.246988,0.259036,0.313253,0.307229,0.373494,0.135542,NaN
2011-02-01,1.0,0.183544,0.180380,0.262658,0.268987,0.234177,0.253165,0.262658,0.272152,0.300633,0.088608,NaN,NaN
2011-03-01,1.0,0.162371,0.257732,0.195876,0.213918,0.172680,0.252577,0.219072,0.275773,0.097938,NaN,NaN,NaN
2011-04-01,1.0,0.192157,0.203922,0.192157,0.184314,0.203922,0.219608,0.231373,0.066667,NaN,NaN,NaN,NaN
2011-05-01,1.0,0.160643,0.172691,0.144578,0.208835,0.232932,0.244980,0.088353,NaN,NaN,NaN,NaN,NaN
2011-06-01,1.0,0.159420,0.125604,0.198068,0.236715,0.299517,0.091787,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,1.0,0.161850,0.179191,0.219653,0.254335,0.098266,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,1.0,0.215827,0.201439,0.251799,0.100719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# retention_percentage_table = retention_percentage_table*100
# retention_percentage_table.apply(lambda x: round(x,2)).reset_index()
# retention_percentage_table

### 1.4.2. Cumulative Lifetime Revenue

#### Formula


### 1.4.3. Customer Lifetime Revenue

#### Definition

- Customer lifetime revenue (CLR) is the total amount of money a customer spends on a business's products or services over their entire relationship. It's also known as customer lifetime value (CLTV).

#### Formula

- $CLR\ |\ CLV = \frac{ARPU\ \times\ Gross\ Margin}{Churn\ Rate}$

    + $ARPU = \frac{Total\ Revenue}{Total\ Number\ Of\ Customers}$

    + $Gross\ Margin = \frac{Revenue\ -\ COGS}{Revenue}$ 

        + For total $COGS = Beginning\ Inventory\ + Purchases\ - Ending\ Inventory$
        
        + For per customer $COGS = \sum{(Unit\ Cost\ Of\ Item \times\ Quantity\ Purchased\ per\ Item)}$

    + $Churn\ Rate = \frac{Total\ Number\ Of\ Churned\ Customers}{Total\ Number\ Of\ Customers}$

#### Theory Example

If a company has a gross margin of 20%, an average revenue per user of $100, and a churn rate of 5%, then the CLV is $400: 
- CLV = (100 * 0.2) / 0.05 = 400

### 1.4.4. Net Revenue 

### 1.4.5. Net Revenue Retention

## 1.5. Visualizing Cohort Analysis
Each row in the heatmap represents a cohort and visualizes the percentage of users retained over time.

In [49]:
def plot_cohorts_heatmap(dataframe: pd.DataFrame) -> plt.Figure:
    cohort_size = dataframe.iloc[:,0]
    dataframe = dataframe.divide(
        other=cohort_size
        ,axis=0
    )[::-1].fillna(0)
    ylabel = [str(dt.date(int(i.split('-')[0]), int(i.split('-')[1]), int(i.split('-')[2])).strftime(format='%B, %Y')) for i in dataframe.index]

    fig = ff.create_annotated_heatmap(
        z = dataframe.values
        , annotation_text = dataframe.map(lambda x: '{:.1%}'.format(x) if x > 0 else '').values.tolist()
        , y = ylabel
        , x = ['Month '+ str(int(i)+1) for i in dataframe.columns]
        , showscale = True
    )

    fig.update_layout(
        width=1000
        , height=700
        , xaxis={"title": "# Periods Elapsed"}
        , font_color = 'rgb(255,255,255)'
        , title="User Retention Rate by Cohort: Heatmap"
        , paper_bgcolor='rgb(0,0,0)'
    )

    fig.show()

plot_cohorts_heatmap(cohort_counts)

# II. Case Study 